In [1]:
import os
import time
import datetime
import numpy as np
import pandas as pd
from datetime import datetime, date

In [2]:
eventos = pd.read_pickle('/kaggle/input/SuperEventos_Criticidad_AguasAbajo.pkl')
# Crear la nueva columna como un DataFrame separado
nueva_columna = pd.DataFrame({
    'TRAMOS_AGUAS_ABAJO_CODES': pd.Series([0] * len(eventos), dtype='object')
}, index=eventos.index)

# Concatenar la nueva columna al DataFrame original
eventos = pd.concat([eventos, nueva_columna], axis=1)
redmt = pd.read_pickle('/kaggle/input/REDMT_1.pkl')

In [3]:
e = 5
inicio, fin = round(len(eventos)/5*(e-1)), round(len(eventos)/5*(e))
eventos_1 = eventos.iloc[inicio:fin]

In [4]:
# Tiempo de inicio del proceso
inicio_tiempo = time.time()

# Número total de filas
total = len(eventos_1)
inicio_str = inicio.strftime("%Y-%m-%d") if isinstance(inicio, pd.Timestamp) else inicio
fin_str = fin.strftime("%Y-%m-%d") if isinstance(fin, pd.Timestamp) else fin

print(f"Procesando eventos del {inicio_str} hasta {fin_str}...\n")

porcentaje_anterior = 0  # Para rastrear el último porcentaje impreso

for i, (index, row_1) in enumerate(eventos_1.iterrows(), start=1):
    # Procesamiento
    nodos_aguas_abajo = set()
    puntos_tramos = eval(row_1['TRAMOS_AGUAS_ABAJO'])  # Usar directamente row_1
    nodos_aguas_abajo.update(puntos_tramos)

    redmt_seleccionado = redmt.loc[(redmt['FECHA'].dt.year == row_1['inicio'].year) & 
                                   (redmt['FECHA'].dt.month == row_1['inicio'].month) & 
                                   (redmt['MUN'] == row_1['MUN'])]

    codes = set()

    for _, row_2 in redmt_seleccionado.iterrows():
        if ((row_2['LATITUD'], row_2['LONGITUD']) in nodos_aguas_abajo and 
            (row_2['LATITUD2'], row_2['LONGITUD2']) in nodos_aguas_abajo):
            codes.add(row_2['CODE'])

    eventos_1.at[index, 'TRAMOS_AGUAS_ABAJO_CODES'] = str(codes)  # Usar at para asignar valores

    # Calcular el porcentaje de progreso
    porcentaje_actual = int((i / total) * 100)  # Porcentaje completado
    if porcentaje_actual > porcentaje_anterior:  # Solo imprimir si cambia el porcentaje
        tiempo_transcurrido = time.time() - inicio_tiempo  # Tiempo desde el inicio

        # Construir la barra de progreso
        barra = f"[{'#' * (porcentaje_actual // 2)}{'.' * (50 - porcentaje_actual // 2)}]"
        
        # Mostrar barra de progreso con porcentaje, filas procesadas y tiempo transcurrido
        print(f"\r{barra} {porcentaje_actual}% ({i}/{total} filas completadas) - {tiempo_transcurrido:.2f} s")
        
        porcentaje_anterior = porcentaje_actual

print("\nProcesamiento completado.")

Procesando eventos del 133058 hasta 166323...

[..................................................] 1% (333/33265 filas completadas) - 225.96 s
[#.................................................] 2% (666/33265 filas completadas) - 437.22 s
[#.................................................] 3% (998/33265 filas completadas) - 649.67 s
[##................................................] 4% (1331/33265 filas completadas) - 867.96 s
[##................................................] 5% (1664/33265 filas completadas) - 1085.47 s
[###...............................................] 6% (1996/33265 filas completadas) - 1300.84 s
[###...............................................] 7% (2329/33265 filas completadas) - 1508.37 s
[####..............................................] 8% (2662/33265 filas completadas) - 1717.02 s
[####..............................................] 9% (2994/33265 filas completadas) - 1928.12 s
[#####.............................................] 10% (3327/33265 

In [5]:
eventos_1.to_pickle(f'Eventos_AguasAbajo_CODEs_{e}.pkl')